In [6]:
import tarfile
import re
from datetime import datetime
from collections import namedtuple, Counter

 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import altair as alt
import re
from _datetime import date
from textblob import TextBlob
import time

 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re



In [7]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['len'] = np.array([len(tweet.text) for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        df['source'] = np.array([tweet.source for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df


In [8]:
tar = tarfile.open("enron_mail_20150507.tar.gz", "r")

In [9]:
items = tar.getmembers()

In [10]:
Email = namedtuple('Email','Date, From, To, Subject, Cc, Bcc, Message')

In [11]:
def get_msg(item_number):

    f = tar.extractfile(items[item_number])

    try: 
        date = from_ = to = subject = cc = bcc = message = ''
        in_to = False
        in_message = False
        to = []
        message = []
        item = f.read().decode()
        item = item.replace('\r','').replace('\t','')
        lines = item.split('\n')
        # print(Len(Lines))
        for num, line in enumerate(lines):

            if line.startswith('Date:') and not date:
                date = datetime.strptime(' '.join(line.split('Date: ')[1].split()[:-2]),'%a, %d %b %Y %H:%M:%S') 
            elif line.startswith('From: ') and not from_:
                from_ = line.replace( 'From: ', '').strip()
            elif line.startswith('To: ') and not to:
                in_to = True
                to = line.replace('To: ', '' ).replace(',', '').replace( ';', '').split()
            elif line.startswith('Subject:') and not subject:
                in_to = False
                subject = line.replace('Subject:', '').strip()
            elif  line.startswith('Cc:') and not cc:
                cc = line.replace('Cc:', '').replace(',', '').replace( ';', '').split()
            elif line.startswith( 'Bcc: ' ) and not bcc:
                bcc = line.replace('Bcc: ', ''). replace( ',', '' ).replace( ';', '' ).split()
            elif in_to:
                to.extend(line.replace(',', '').split())
            elif line.startswith('Subject:') and not subject:
                in_to = False
            elif line.startswith('X-FileName'):
                in_message = True
            elif in_message:
                message.append(line)

        to = '; '.join(to).strip()
        cc = '; '.join(cc).strip()
        bcc = '; '.join(bcc).strip()
        message = ' '.join(message).strip()
        email = Email(date, from_, to, subject, cc, bcc, message)

        return email

    except Exception as e: 
        return e 



In [12]:
msg = get_msg(3)



In [13]:
msg.Message

'NameNew TitleEffective DateMid Year promotion Yes/No  Floyd, JodieSr Cust Svc Rep (no change)7/16/01No  Buehler, CraigSr Mkt/Sup Analyst (no change)7/16/01No  Wagoner, MikeTeam Advisor - Gas Control7/1/01No  Clapper, KarenSr Cust Svc Rep8/1/01Yes  Greaney, ChrisSr Cust Svc Rep8/1/01Yes  Wilkens, JerrySr Cust Svc Rep8/1/01Yes  Minton, KevinPipeline Controller8/1/01Yes  Cox, DonPipeline Controller8/1/01Yes  Hanagriff, RichardSr Accounting Control Spec8/1/01Yes   Thanks, MS'

In [14]:
len(items)

520901

In [15]:
dates = []

for num in range(len(items)):
    msg = get_msg(num)
    if not isinstance(msg, tuple):
        continue
    dates.append(msg.Date)

In [16]:
dates_count = Counter(dates)


In [17]:
df = pd.DataFrame.from_dict(dates_count, orient = 'index').reset_index()
df.columns = ['datetime', 'count']

In [18]:
df['date'] = df['datetime'].map(lambda x: x.date())

In [19]:
df.head()

,datetime,count,date
0,2001-09-14 14:05:43,1,2001-09-14
1,2001-09-10 10:33:15,1,2001-09-10
2,2001-08-20 12:34:19,1,2001-08-20
3,2001-07-05 08:35:29,2,2001-07-05
4,2001-07-26 08:16:14,1,2001-07-26


In [20]:
pivot = df.pivot_table(index = 'date', values = 'count', aggfunc = 'sum').reset_index()
pivot = pivot[(pivot['date'] > pd.datetime(2000, 1, 1).date()) & (pivot ['date'] < pd.datetime(2002, 6, 30).date())]
pivot['date'] = pd.to_datetime(pivot['date'])

In [21]:
alt.Chart(pivot).mark_bar().encode(
    x = 'date',
    y = 'count',
    tooltip = ['date', 'count'],
).properties(width = 800)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [22]:
df 

,datetime,count,date
0,2001-09-14 14:05:43,1,2001-09-14
1,2001-09-10 10:33:15,1,2001-09-10
2,2001-08-20 12:34:19,1,2001-08-20
3,2001-07-05 08:35:29,2,2001-07-05
4,2001-07-26 08:16:14,1,2001-07-26
...,...,...,...
224065,2000-09-25 02:41:00,1,2000-09-25
224066,2001-03-21 03:29:00,1,2001-03-21
224067,2000-08-09 04:57:00,1,2000-08-09
224068,2000-08-15 05:30:00,1,2000-08-15


In [23]:
msgFROM = []
msgMESSAGE = []
msgDATE = []


for num in range(len(items)):
   msg = get_msg(num)
   if not isinstance(msg, tuple):
       continue
   msgFROM.append(msg.From)
   msgMESSAGE.append(msg.Message)
   msgDATE.append(msg.Date)

dfALL = pd.DataFrame(list(zip(msgFROM, msgDATE, msgMESSAGE)), columns=['from', 'date', 'message'])
dfALL.head(10)

,from,date,message
0,fran.fagan@enron.com,2001-09-14 14:05:43,NameNew TitleEffective DateMid Year promotion ...
1,maggie.matheson@enron.com,2001-09-10 10:33:15,First I want to thank you for all of your help...
2,sharon.brown@enron.com,2001-08-20 12:34:19,Below listed is the contact information for Th...
3,lynn.blair@enron.com,2001-07-05 08:35:29,"Sharon, could add the following company/person..."
4,sharon.brown@enron.com,2001-07-26 08:16:14,Attached is a draft of the agenda for the Wint...
5,sharon.brown@enron.com,2001-10-03 12:28:17,Attached are two follow-up documents from Shel...
6,gerry.medeles@enron.com,2001-09-27 06:37:23,After 2:00 will work with me. Attached is doc...
7,bradley.holmes@enron.com,2001-10-02 12:29:55,Attached is the prioritization list of the sys...
8,sharon.brown@enron.com,2001-08-16 12:23:57,Thank you for responding to our invitation. A...
9,maggie.matheson@enron.com,2001-09-13 10:06:38,Below is a list of the feedback given to us by...


In [24]:

tweet_analyzer = TweetAnalyzer()

dfALL['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in dfALL['message']])

try:
    dfALL['date'] = dfALL['date'].map(lambda x: x.date())
except Exception:
    print(1)


messagedfREDACTED = dfALL[dfALL['sentiment'] == -1]

messagedfREDACTED['sentiment'] = messagedfREDACTED['sentiment'] * -1

messagedfREDACTED



,from,date,message,sentiment
21,steve.kirk@enron.com,2001-12-10,The call-in number for the December 12th (Wedn...,1
29,kay.threet@enron.com,2001-10-18,"Mike, Kent and Tim called me this morning and...",1
32,john.buchanan@enron.com,2001-06-11,FYI ---------------------- Forwarded by John B...,1
34,bradley.holmes@enron.com,2001-08-03,The following individuals have been identified...,1
36,bradley.holmes@enron.com,2001-08-07,You have been identified to participate in the...,1
...,...,...,...,...
517269,larry.campbell@enron.com,2001-09-21,NORTHERN NATURAL GAS COMPANY Submitted Title ...,1
517272,larry.campbell@enron.com,2001-07-03,In the final phase of developing and determini...,1
517281,larry.campbell@enron.com,2000-09-08,---------------------- Forwarded by Larry F Ca...,1
517295,larry.campbell@enron.com,2000-08-16,---------------------- Forwarded by Larry F Ca...,1


In [25]:
pivot = messagedfREDACTED.pivot_table(index = 'date', values = 'sentiment', aggfunc = 'sum').reset_index()
pivot = pivot[(pivot['date'] > pd.datetime(2000, 1, 1).date()) & (pivot ['date'] < pd.datetime(2002, 6, 30).date())]
#pivot = pivot[(pivot['date'])]
pivot['date'] = pd.to_datetime(pivot['date'])

In [26]:
alt.Chart(pivot).mark_bar().encode(
    x = 'date',
    y = 'sentiment',
    tooltip = ['date', 'sentiment'],
).properties(width = 800)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [27]:
# UniqueEmails = messagedf['author'].unique().tolist()

# df_dict = {name: messagedf.loc[messagedf['author'] == name] for name in UniqueEmails}


# df_dict['lynn.blair@enron.com']



In [28]:

messagedfREDACTEDPOS = dfALL[dfALL['sentiment'] == 1]


messagedfREDACTEDPOS

,from,date,message,sentiment
0,fran.fagan@enron.com,2001-09-14,NameNew TitleEffective DateMid Year promotion ...,1
1,maggie.matheson@enron.com,2001-09-10,First I want to thank you for all of your help...,1
2,sharon.brown@enron.com,2001-08-20,Below listed is the contact information for Th...,1
3,lynn.blair@enron.com,2001-07-05,"Sharon, could add the following company/person...",1
6,gerry.medeles@enron.com,2001-09-27,After 2:00 will work with me. Attached is doc...,1
...,...,...,...,...
517306,louis.soldano@enron.com,2000-11-07,it's possible that we could get pulled into th...,1
517307,george.robinson@enron.com,2001-04-12,Larry- Two files are attached. The first file ...,1
517308,george.robinson@enron.com,2001-02-14,We have received the water well survey for the...,1
517309,george.robinson@enron.com,2000-08-11,Roger- We will require off-site access for the...,1


In [29]:
pivot = messagedfREDACTEDPOS.pivot_table(index = 'date', values = 'sentiment', aggfunc = 'sum').reset_index()
pivot = pivot[(pivot['date'] > pd.datetime(2000, 1, 1).date()) & (pivot ['date'] < pd.datetime(2002, 6, 30).date())]
#pivot = pivot[(pivot['date'])]
pivot['date'] = pd.to_datetime(pivot['date'])

In [30]:
alt.Chart(pivot).mark_bar().encode(
    x = 'date',
    y = 'sentiment',
    tooltip = ['date', 'sentiment'],
).properties(width = 800)

<VegaLite 3 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
